In [39]:
#import required libraries
from azure.ai.ml import MLClient, command
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

# specify the details of your subscription
SUBSCRIPTION_ID = "e5615bfe-b43b-41ce-bccb-b78867c2ce63"
RESOURCE_GROUP = "rg-dp100-demo-001"

# get a handle to the subscription
load_dotenv("python.env")

True

### Listing all workspaces

In [40]:

ml_client = MLClient(DefaultAzureCredential(), 
                     subscription_id=SUBSCRIPTION_ID, 
                     resource_group_name=RESOURCE_GROUP)

In [41]:
# getting the workspace
print([ws.name for ws in ml_client.workspaces.list()])
ws = ml_client.workspaces.get("mlw-dp100-demo")
ws.name

['mlw-dp100-demo']


'mlw-dp100-demo'

### Getting assets of a workspace

#### Environments - get existing env

In [46]:
# in order to list the environments, which is an attribute of the MLClient object,
# but the environment is an object under a specific workspace in the MLClient
# so we need to specify the workspace name in the MLClient handle
ml_client = MLClient(DefaultAzureCredential(), 
                     subscription_id=SUBSCRIPTION_ID, 
                     resource_group_name=RESOURCE_GROUP,
                     workspace_name="mlw-dp100-demo")
envs = ml_client.environments.list()
# we can print the name and latest version of the environments
print([(env.name, env.latest_version) for env in envs])

[('testenv', '1'), ('AzureML-AI-Studio-Development', '1'), ('AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu', '10'), ('AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu', '14'), ('AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu', '14'), ('AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu', '14'), ('AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu', '17'), ('AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu', '7'), ('AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu', '9'), ('AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu', '27'), ('AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu', '26'), ('AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu', '26'), ('AzureML-sklearn-1.0-ubuntu20.04-py38-cpu', '36'), ('AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu', '36'), ('AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu', '44'), ('AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu', '43'), ('AzureML-sklearn-0.24-ubuntu18.04-py37-cpu', '47'), ('AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu', '51'), ('AzureML-pytorch-1.7-ubuntu

In [47]:
# getting the actual environment using the name and version
env = ml_client.environments.get(name="testenv", version=1)
env

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'testenv', 'description': 'from AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:47', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/e5615bfe-b43b-41ce-bccb-b78867c2ce63/resourceGroups/rg-dp100-demo-001/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-demo/environments/testenv/versions/1', 'Resource__source_path': None, 'base_path': 'd:\\Repositories\\GitHub\\dp-100', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000002157CA0AC10>, 'serialize': <msrest.serialization.Serializer object at 0x000002157C36EED0>, 'version': '1', 'latest_version': None, 'conda_file': None, 'image': None, 'build': <azure.ai.ml.entities._assets.environment.BuildContext object at 0x000002157A7BC6D0>, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'con

#### Computes

In [31]:
# we can list the compute clusters created:
# note that same as above, we need to init the MLClient with the workspace name
# else the ValueError: No value for given attribute will be thrown
print([(compute.name, compute.provisioning_state) for compute in ml_client.compute.list()])

# we can get a specific compute cluster by name:
# cluster = ml_client.compute.get("vmcluster-ml-dev-002")
# cluster.provisioning_state

[('vm-ml-dev', 'Succeeded'), ('vmcluster-ml-dev', 'Succeeded'), ('vmcluster-ml-dev-002', 'Succeeded')]


### Submit job to a compute cluster

In [35]:
# define the command
# using the env object and the compute cluster name
# note that the command is the actual bash command to be executed
command_job = command(
    code="./src", # The source code to run the job. Can be a local path or remote path
    command="python hello.py",
    environment=env, # must be an instance of the Environment class
    compute="vmcluster-ml-dev-002",
)

returned_job = ml_client.jobs.create_or_update(command_job)
returned_job

Uploading src (0.0 MBs): 100%|##########| 109/109 [00:00<00:00, 1212.77it/s]




Experiment,Name,Type,Status,Details Page
dp-100,dreamy_bucket_hmdgd19wdg,command,Starting,Link to Azure Machine Learning studio


In [38]:
command_job = command(
    name="test-job", # if None a random name will be generated
    code="./src", # The source code to run the job. Can be a local path or remote path
    command="python hello.py",
    environment=env, # must be an instance of the Environment class
    compute="vmcluster-ml-dev-002",
)

returned_job = ml_client.jobs.create_or_update(command_job)
returned_job

Experiment,Name,Type,Status,Details Page
dp-100,test-job,command,Starting,Link to Azure Machine Learning studio
